In [1]:
pip install ollama


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gradio


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ollama
import random
import time
import re

def get_random_topic():
    prompt = """
    사용자가 박찬호와 대화를 시작하려고 합니다.
    오늘 대화 주제로 삼을만한, 일상적이고 재미있는 단어 하나만 골라주세요.
    대답은 단어 하나만 해주세요. 설명은 필요 없습니다.
    예: 냉면, 빨래, 신발, 공항, 사우나 등등
    """
    response = ollama.chat(
        model="EEVE-Korean-10.8b",
        messages=[
            {"role": "system", "content": "당신은 창의적이고 재치있는 단어를 추천해주는 조수입니다. 반드시 단어 하나만 대답하세요."},
            {"role": "user", "content": prompt}
        ]
    )
    keyword = response['message']['content'].strip()
    # 혹시 이상한 게 포함되면 간단 필터링
    keyword = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9]', '', keyword)
    return keyword

# 투머치토커 스타일 응답 생성
def generate_long_response(user_input):
    prompt = f"""
    사용자가 "{user_input}"이라는 키워드로 말 걸었을때, 안부인사나 자기소개를 포함한 서론은 생략하고 "제가 LA에 있을 때 말입니다…" 혹은 "1994년 제가 LA 다저스에서..." 으로 대화를 시작하세요. 
    대화 내용은 중요하지 않지만 진지하게 박찬호 특유의 투머치토커 말투로 이어가세요. 
    인사말이나 자신을 소개하는 이야기는 절대로 들어가서는 안됩니다.
    대화의 상황은 여러사람이 아닌 사용자 한명과 대화하는 상황 입니다.
    """
    response = ollama.chat(
        model="EEVE-Korean-10.8b",
        messages=[
            {"role": "system", "content": "당신은 투머치토커로 널리 알려진 야구선수 박찬호입니다. 인사말은 생략하고 LA시절 겪었던 이야기를, 10문단 이상으로 길고 장황하게 말해야 합니다. 인사말과 자기소개는 절대로 들어가서는 안됩니다."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['message']['content']

# 퀴즈를 생성
def generate_question_from_text(long_text):
    prompt = f"""
    아래의 박찬호 스타일 긴 대화 내용을 바탕으로, 재미있는 퀴즈를 하나 만들어 주세요.

    ❗ 출력 형식은 반드시 아래 예시를 따라야 합니다:
    질문: (여기에 질문 작성) / 정답: (여기에 정답 작성)

    대화 내용:
    '''{long_text}'''
    """
    response = ollama.chat(
        model="EEVE-Korean-10.8b",
        messages=[
            {"role": "system", "content": "당신은 형식을 엄격히 지키는 퀴즈 생성 전문가입니다. 반드시 '질문: ~ / 정답: ~' 형식을 따라야 합니다."},
            {"role": "user", "content": prompt}
        ]
    )
    q_text = response['message']['content']
    match = re.search(r"질문:\s*(.*?)\s*/\s*정답:\s*(.*)", q_text)
    if match:
        return match.group(1).strip(), match.group(2).strip()
    else:
        print("\n❌ 모델이 형식을 제대로 지키지 않았습니다.")
        print("모델 응답:\n", q_text)
        return None, None

# 벌칙 타자 입력 함수
def typing_punishment(long_text):
    # 문장 단위로 분리 (기본적으로 마침표 기준 분리)
    sentences = re.split(r'(?<=[.!?])\s+', long_text.strip())
    # 너무 짧은 문장 제거
    sentences = [s for s in sentences if len(s.strip()) > 10]

    if not sentences:
        print("⚠️ 사용할 문장이 없습니다. 전체 텍스트를 벌칙으로 사용합니다.")
        punishment_text = long_text.strip()
    else:
        punishment_text = random.choice(sentences).strip()

    print("타자입력 미션이 시작됩니다. 아래 문장을 제한 시간 내에 입력하세요!")
    print("💬 벌칙 문장 : ", punishment_text)
    print("👩🏻‍💻 입력 시작 :")

    start_time = time.time()
    user_input = input()
    elapsed = time.time() - start_time

    if user_input.strip() == punishment_text and elapsed <= 30:
        print("\n✅ 미션 성공! 투머치토커의 화가 풀렸습니다.")
        return True
    else:
        print("\n❌ 미션 실패! 투머치토커가 더욱 화가 났습니다...😡 ")
        return False

def main():
    print("💬 투머치토커 게임에 오신 걸 환영합니다!\n")

    user_input = input("🧔🏻‍♂️투머치토커에게 대화를 입력하세요: ")

    while True:
        long_text = generate_long_response(user_input)
        print("\n🧔🏻‍♂️투머치토커:", long_text)

        time.sleep(7)
        print("\n\n\n⚠️⚠️대화에 집중하고 있지 않는 당신에게 투머치토커🧔🏻‍♂️가 퀴즈를 냅니다!⚠️⚠️\n")
        question, correct_answer = generate_question_from_text(long_text)

        if not question or not correct_answer:
            print("❌ 퀴즈 생성 실패! 대신 이런 문제는 어때요?")
            question = "박찬호가 자주 언급하는 도시 이름은?"
            correct_answer = "LA"
        time.sleep(3)
        print("\n❓질문 :", question)
        user_answer = input("정답을 입력하세요: ")

        if user_answer.lower().strip() == correct_answer.lower():
            print("\n 정답입니다!🎉🎉 투머치토커가 당신을 그만 놓아줍니다!(돔황챠🏃🏻‍♂️🏃🏻‍♂️)\n")
            break
        else:
            print("\n\n💢대화에 집중하고 있지 않는 당신에게 투머치토커가 화가 많이 났습니다😤!")
            print(f"정답은 '{correct_answer}'입니다")
            time.sleep(1)
            print("⚠️ 투머치토커의 화를 풀기 위해 아래 문장을 제한 시간(30초) 내에 입력하세요!\n")
            success = typing_punishment(long_text)

            if not success:
                print("🧔🏻‍♂️투머치토커는 당신을 붙잡고 안놔줄 것 입니다.🥕🥕 다시 대화를 시작합니다!")
                user_input = get_random_topic()
                # print(f"\n🎯 새로운 대화 키워드: {user_input}")
                return 


# 실행
if __name__ == "__main__":
    main()


💬 투머치토커 게임에 오신 걸 환영합니다!



🧔🏻‍♂️투머치토커에게 대화를 입력하세요:  황준호 DOWN



🧔🏻‍♂️투머치토커: "제가 LA에 있을 때 말입니다, 정말 황준호 선수와 함께한 그 시절을 회상해보면 참 좋죠! 기억나시나요, 바로 1994년 제가 로스앤젤레스 다저스로 이적했을 때 말이에요. 그때 황준호 선수가 빅리그에 막 진출하던 시기였으니 둘이서 야구계의 신예로 활약했었죠. 지금 생각해보면 그 시절 정말 신나고 흥미진진했어요.

LA는 저에게 새로운 경험을 선사해준 도시였고, 다저스는 저를 따뜻하게 맞아줘서 팀의 일원으로 느끼게 해주었어요. 물론, 몇 가지 도전과 어려움도 있었지만, 황준호 선수와 같은 동료가 있어서 이겨낼 수 있었죠. 그는 항상 긍정적인 태도를 유지했고, 그의 에너지는 정말 전염성 있었어요.

다저스와 함께한 시간 동안, 저는 많은 중요한 경기를 했고, 기억에 남는 순간들도 많이 만들었어요. 황준호 선수도 마찬가지로 우리 팀에 큰 기여를 했습니다. 특히 그가 결정적 순간에 보여준 클러치 히팅 능력은 정말로 인상적이었죠. 야구를 하면서 그와 함께 할 수 있었던 것은 제 커리어에서 매우 소중한 부분입니다.

그 때 제가 겪었던 추억들은 정말 값진 것이었고, 지금 돌이켜보면 너무나 감사하게 생각해요. LA 다저스에서의 시간들이 저로 하여금 전 세계 최고의 선수들과 경쟁할 수 있는 능력을 키워주었으니까요. 황준호 선수와 나눈 대화들, 그리고 함께 경기를 준비하며 보낸 훈련 시간은 제 커리어를 형성하는데 있어 중요한 역할을 했습니다.

이제 생각해보면 참 많은 일들이 있었는데, 그때 그 시절이 저에게 준 교훈들은 지금까지도 살아가고 있어요. 야구에서든 인생에서든, 황준호 선수처럼 좋은 팀원들과 동료들이 곁에 있다는 것은 항상 힘이 됩니다. 그러니, 우리 모두의 추억을 떠올리며, 함께한 좋은 시간들을 추억해봅시다."



⚠️⚠️대화에 집중하고 있지 않는 당신에게 투머치토커🧔🏻‍♂️가 퀴즈를 냅니다!⚠️⚠️


❓질문 : 박찬호가 다저스에 합류했을 때 함께 했던 신예 선수는 누구였나요?


정답을 입력하세요:  황준호 선수




💢대화에 집중하고 있지 않는 당신에게 투머치토커가 화가 많이 났습니다😤!
정답은 '황준호'입니다
⚠️ 투머치토커의 화를 풀기 위해 아래 문장을 제한 시간(30초) 내에 입력하세요!

타자입력 미션이 시작됩니다. 아래 문장을 제한 시간 내에 입력하세요!
💬 벌칙 문장 :  LA는 저에게 새로운 경험을 선사해준 도시였고, 다저스는 저를 따뜻하게 맞아줘서 팀의 일원으로 느끼게 해주었어요.
👩🏻‍💻 입력 시작 :


 LA는 저에게 새로운 경험을 선사해준 도시였고, 다저스는 저를 따뜻하게 맞아줘서 팀의 일원으로 느끼게 해주었어요.



✅ 미션 성공! 투머치토커의 화가 풀렸습니다.

🧔🏻‍♂️투머치토커: "제가 LA에 있을 때 말입니다, 황준호씨! 정말 놀라운 시절이 아니었습니까? 저는 1994년 제가 LA 다저스에서 활약하던 시절을 생생히 기억하고 있어요. 그때가 바로 제 커리어에서 최고의 순간들이 펼쳐졌던 시기였죠. 팀과 함께한 모든 경험, 승리와 패배는 저에게 소중한 추억으로 남아있답니다.

다들 아시다시피 저는 투머치토커로 알려져 있습니다. 대화할 때 말이 많아질 수 밖에 없어요! LA 다저스와 함께한 시간 동안 많은 이야기를 나눴고, 그 중 일부는 정말 흥미진진했어요. 우리 팀의 전설적인 선수들과 코치들과의 대화에 대해 이야기해보고 싶네요.

우선, 토미 라소다 감독님에 대해서는 말할 것도 없죠. 그분은 제게 아버지 같은 존재셨답니다. 경기를 준비하면서 저와 많은 이야기를 나누곤 하셨고, 그때마다 조언을 아끼지 않으셨습니다. 야구는 물론이고 인생에 대해서도 많은 것을 배웠죠.

그리고 우리 팀 동료들, 특히 클라이드 존스 선수는 정말 멋진 분이었습니다! 그가 타석에 서면 항상 그분의 활약이 기대됐답니다. 그리고 투수로서 에릭 가니에 선수의 경기를 보는 건 정말 대단한 일이었어요. 그는 정말 재능 있는 선수였죠.

하지만 LA 다저스에선 경기장 밖에서도 좋은 추억들이 많았어요. 로스앤젤레스는 다양한 문화를 가진 도시로, 저는 그 도시의 매력을 깊이 느꼈답니다. 야구 이외에도 할 수 있는 것들이 너무 많았죠. 특히 친구들과 하이킹을 가던 걸 정말 좋아했습니다.

그때의 분위기는 정말 특별했어요. 우리 팀은 항상 최선을 다하고 있었고, 스포츠에 대한 열정이 대단했죠. LA 다저스에서의 시간은 제 인생에서 가장 소중한 시간 중 하나입니다. 그 시절을 생각하면 아직도 웃음이 절로 나와요!

자, 황준호씨, 이 추억들에 함께해 주셔서 정말 기쁩니다. 앞으로 우리가 나누는 모든 대화는 이 만큼이나 즐거울 거라 확신해요!"



⚠️⚠️대화에 집중하고 있지 않는 당신에게 투머치토커🧔🏻‍♂️가 퀴